# Download Torrent to Google Drive

### 1. Install libtorrent Dependency

In [ ]:
!apt install -y python3-libtorrent

import libtorrent as lt

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

### 2. Mount Google Drive

In [ ]:
from google.colab import drive, files

drive.mount("/content/drive")

## 3. Add Torrent Source



In [ ]:
# Add torrents from multiple magnet links or .torrent files
downloads = []

while True:
    source = input("Enter a magnet link or type 'file' to upload a .torrent file or 'exit' to stop: ")

    if source.lower() == 'exit':
        break
    elif source.lower() == 'file':
        uploaded = files.upload()
        info = lt.torrent_info(list(uploaded.keys())[0])
        h = ses.add_torrent({'ti': info, 'save_path': '/content/drive/MyDrive/Torrents'})
    else:
        params = lt.parse_magnet_uri(source)
        params.save_path = '/content/drive/MyDrive/Torrents'
        h = ses.add_torrent(params)

    downloads.append(h)

    print('\nstarting', h.status().name)

## 4. Download File

In [ ]:
import time
from IPython.display import clear_output

# Monitor downloads
while downloads:
    # Clear the output before printing the status
    clear_output(wait=True)

    next_iteration_downloads = []

    for download in downloads:
        s = download.status()

        state_str = [
            'queued',
            'checking',
            'downloading metadata',
            'downloading',
            'finished',
            'seeding',
            'allocating',
            'checking fastresume'
        ]

        print('%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s %s' % (
            s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
            s.num_peers, state_str[s.state], s.name))

        if not download.status().is_seeding:
            next_iteration_downloads.append(download)
        else:
            print('\n', download.status().name, 'complete')

    downloads = next_iteration_downloads

    time.sleep(1)